# Karan PDFs: the real deal

In [256]:

# Karan_test_version1_not_searched_the_description = pd.read_csv(r'fullAnonDF.csv', 
#                          index_col='IDP',
#                         )



Karan_test = pd.read_csv(r'fullAnonDF_vs2.csv',   # Karan: "133 extra features found across the 5000 files"
#                          index_col='IDP',
                        )

In [257]:
Karan_test.head()


# problems:
# 1. too few automatisms (none of oral, LL, UL)
# 2. AUTOMATISMS does not equal True when other automatisms is True - this is a mistake.

,Unnamed: 0,IDP,Had surgery,OP Date,OP Type,Side,IC,Entirely Seizure-Free,pdfSequenceArray,mlSequenceArray,...,Aphasia,Dysphasia,Pallilalia,Aphemia,Corprolalia,Dialeptic/LOA,LOC,EZ,AUTOMATISMS,Hippocampal Sclerosis
0,0,1500,False,NaN,NaN,NaN,NaN,NaN,"[[60, 55]]","[[49, 17]]",...,False,False,False,False,False,True,False,False,False,True
1,1,1501,False,NaN,NaN,NaN,NaN,NaN,"[[3, 50]]","[[7, 17]]",...,False,False,False,False,False,False,False,False,False,False
2,2,1502,False,NaN,NaN,NaN,NaN,NaN,"[[19, 17, 20], [55]]","[[9, 2, 46], [17]]",...,False,False,False,False,False,False,False,False,False,False
3,3,1503,False,NaN,NaN,NaN,NaN,NaN,"[[17, 15, 16, 20, 60], [62], [20]]","[[2, 4, 4, 46, 49], [26], [46]]",...,False,False,False,False,False,False,False,False,False,True
4,4,1504,False,NaN,NaN,NaN,NaN,NaN,"[[60], [37]]","[[49], [13]]",...,False,False,False,False,False,False,False,False,False,True


In [258]:
# 1. get the ESF
Karan_ESF= Karan_test.loc[Karan_test[ESF]==True]
Karan_ESF.shape  # 86

(86, 61)

In [259]:
# 2. check for duplicates, remove the last duplicates
print(Karan_ESF.IDP.nunique())
Karan_ESF.loc[Karan_ESF['IDP'].duplicated()]
Karan_ESF_xduplicated = Karan_ESF.drop(index=Karan_ESF.loc[Karan_ESF['IDP'].duplicated()].index, axis=0)
# Karan_ESF.loc[~Karan_ESF['IDP'].duplicated()  # equivalent
Karan_ESF_xduplicated.shape

67


(67, 61)

In [260]:
(Karan_ESF.drop(index=Karan_ESF.loc[Karan_ESF['IDP'].duplicated()].index, axis=0).all() == Karan_ESF.loc[~Karan_ESF['IDP'].duplicated()].all()).all()

True

In [261]:
# 3. Remove Hx/CCx in OP Type

Karan_ESF_xduplicated = Karan_ESF_xduplicated.drop(index=Karan_ESF_xduplicated.loc[Karan_ESF_xduplicated['OP Type']=='Hx'].index)
Karan_ESF_xduplicated.shape

(66, 61)

In [262]:
# 4. Set EZ target for predictions

Karan_ESF_xduplicated.loc[(Karan_ESF_xduplicated['OP Type'] == 'T Lx'), 'EZ'] = True
Karan_ESF_xduplicated.loc[(Karan_ESF_xduplicated['OP Type'] == 'T Lesx'), 'EZ'] = True

y = Karan_ESF_xduplicated['EZ']

In [263]:
# 5.1  merge AUTOMATISMS and remove others
Karan_ESF_xduplicated['AUTOMATISMS'] = False

Karan_ESF_xduplicated.loc[Karan_ESF_xduplicated['Oral Automatisms']==True, 'AUTOMATISMS'] = True
Karan_ESF_xduplicated.loc[Karan_ESF_xduplicated['Upper Limb Automatism']==True, 'AUTOMATISMS'] = True#
Karan_ESF_xduplicated.loc[Karan_ESF_xduplicated['Other Automatism']==True, 'AUTOMATISMS'] = True
Karan_ESF_xduplicated.loc[Karan_ESF_xduplicated['Lower Limb Automatism']==True, 'AUTOMATISMS'] = True

Karan_ESF_xduplicated.drop(columns=['Oral Automatisms', 'Upper Limb Automatism', 'Other Automatism', 'Lower Limb Automatism'], inplace=True)


In [264]:
# 5.2 remove unused features in training set (no occurence in training set)
train_set_not_seen_features = ['Fencing', 'Lower Limb Automatism', 'Pallilalia', 'Aphemia', 'Drinking', 'Cough', 'Whistling']
train_set_not_seen_features = [i for i in train_set_not_seen_features if i in Karan_ESF_xduplicated.columns]
Karan_ESF_xduplicated.drop(columns=train_set_not_seen_features, inplace=True)


In [265]:
# 6. Set X as only subset of features of above:

if Karan_ESF_xduplicated.columns[9] == 'mlSequenceArray':
    X = Karan_ESF_xduplicated.iloc[:, 10:]
elif Karan_ESF_xduplicated.columns[9] == 'Epigastric':
    X = Karan_ESF_xduplicated.iloc[:, 9:]
# X.drop(columns=HS, inplace=True)   # wil drop later on the fly
X.drop(columns='EZ', inplace=True)
X.shape

(66, 41)

In [267]:

X.head()  # for version2, need to start from 10

,Epigastric,Fear-Anxiety,Psychic,Autonomous-Vegetative,Olfactory-Gustatory,Auditory,Visual,Somatosensory,Vestibular,Non-Specific Aura,...,Ictal Pout,Grimace,Vocalisation,Aphasia,Dysphasia,Corprolalia,Dialeptic/LOA,LOC,AUTOMATISMS,Hippocampal Sclerosis
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
25,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
58,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
69,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [276]:
(X.sum() == 0).sum()

32

In [268]:
# 7. Naive Benchmark:
y.sum()/y.count()

0.8181818181818182

In [269]:
# 8. GB # version 2

GB_RFECV_acc.score(X.drop(columns=HS), y), GB_HS_RFECV_acc.score(X, y)

(0.8181818181818182, 0.8333333333333334)

In [227]:
# 9. SVC(kernel=linear)  # version 1

linear_RFECV_acc.score(X.drop(columns=HS), y), linear_HS_RFECV_acc.score(X, y)

(0.8181818181818182, 0.8484848484848485)

In [270]:
# 9. SVC(kernel=linear)  # version 2

linear_RFECV_acc.score(X.drop(columns=HS), y), linear_HS_RFECV_acc.score(X, y)

(0.8181818181818182, 0.8484848484848485)